In [ ]:
!pip install -U scikit-learn==1.2.2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import xgboost as xgb

In [ ]:
from google.colab import drive
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from google.colab import drive
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/MyDrive/Codigo Tesis Helicobacter Editado/Var_Prev'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

In [ ]:
# Lista para almacenar dataframes
results = []
sm = SMOTE(random_state=42)

for file in csv_files:

    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep=",", header=0, index_col=0)
    df = df.dropna()
    df = df.sample(n=500, replace=True, random_state=42)

    X = df.drop("Prevalencia", axis=1)
    y = df["Prevalencia"]

    # Determinar 'k', el número de variables
    k = X.shape[1]

    # Definir el pipeline con pasos vacíos
    pipeline = Pipeline([
        ('scaler', StandardScaler()),    # Escalado de características
        ('reduce_dim', 'passthrough'),   # Placeholder para la reducción de dimensionalidad
        ('classification', xgb.XGBClassifier (random_state=42))        # Clasificador SVC
    ])

    # Definir los parámetros para GridSearch
    param_grid = [
        {
            'reduce_dim': [SelectKBest(score_func=f_classif),
                SelectKBest(score_func=mutual_info_classif),
                SelectKBest(score_func=chi2)],
            'reduce_dim__k': [k//2,k//4,k//8],
            'classification__max_depth': [3, 5, 7],
            'classification__learning_rate': [0.1,0.15,0.2],
            'classification__n_estimators': [300, 400, 500]
        },
        {
            'reduce_dim': [PCA(random_state=42)],
            'reduce_dim__n_components': [k//2,k//4,k//8],
            'classification__max_depth': [3, 5, 7],
            'classification__learning_rate': [0.1,0.15,0.2],
            'classification__n_estimators': [300, 400, 500]
        }
    ]

    X_res, y_res = sm.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.332, stratify=y_res, random_state=42)

    # Configurar el clasificador con diferentes tamaños de capa oculta
    clf = GridSearchCV(pipeline, param_grid=param_grid, cv=3, scoring='accuracy')

    # Configurar el clasificador con diferentes tamaños de capa oculta
    clf.fit(X_train, y_train)

    test_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, test_pred)

    results.append((file, clf.best_params_, accuracy))

Mejor archivo: Mi25Pre7.csv, Mejor accuracy: 0.92,Mejor Modelo: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 400}


In [ ]:
# Ordenar los resultados por accuracy de mayor a menor
results.sort(key=lambda x: x[2], reverse=True)

# Mostrar el mejor resultado
best_file,best_clasi, best_accuracy = results[0]
print(f"Mejor archivo: {best_file}, Mejor accuracy: {best_accuracy:.2f},Mejor Modelo: {best_clasi}")

Mejor archivo: Mi25Pre7.csv, Mejor accuracy: 0.92,Mejor Modelo: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 400}


In [ ]:
results=pd.DataFrame(results)
results.to_csv('/content/drive/MyDrive/Codigo Tesis Helicobacter Editado/Resultados/XGBClassifier.csv',index=False,sep=';',header=True )